In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# gerais

import pandas as pd
from tqdm import tqdm


# Gensim
from nltk import tokenize
import gensim.corpora as corpora
from gensim.models import CoherenceModel


#plot
import matplotlib
matplotlib.use("TkAgg")  # Define o backend ANTES de importar pyplot

import matplotlib.pyplot as plt  # Só importar depois de definir o backend

import pyLDAvis
import pyLDAvis.gensim_models



import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode


from string import punctuation

from gensim.models import LdaMulticore, CoherenceModel


In [3]:
df = pd.read_csv('../datasets/fakeTelegram.BR_2022.csv')

In [4]:
len(df)

557586

In [5]:
df.columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url', 'trava_zap',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'id_persona',
       'message_type', 'messenger', 'media_name', 'media_md5'],
      dtype='object')

In [6]:
minimo = df['date_message'].min()
maximo = df['date_message'].max()

print(maximo)
print(minimo)

2022-11-11 12:09:48
2022-09-29 00:00:04


In [7]:
df['text_content_anonymous'].head(30)

0     Então é Fato Renato o áudio que eu ouvi no wha...
1     Saiu no YouTube do presidente a 8 horas atrás,...
2     É isso, nossa parte já foi quase toda feita. N...
3              GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA
4                                                   NaN
5     Kķkkkkk to rindo até agora....Quem disse q ia ...
6     *SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...
7                                                   NaN
8                                                   NaN
9     O Deputado Federal pelo NOVO e que foi candida...
10    Saiam desse grupo amigos bolsonaristas, urgent...
11    Gazprom da Rússia: Retomamos o fornecimento de...
12    Saiam desse grupo amigos bolsonaristas, urgent...
13    Saiam desse grupo amigos bolsonaristas, urgent...
14    Saiam desse grupo amigos bolsonaristas, urgent...
15                                                  NaN
16    🚨Olá SEJA bem vindo(a) ..- ao grupo Especuland...
17    Saiam desse grupo amigos bolsonaristas, ur

In [8]:
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Wanda Silva,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Wanda Silva,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Wanda Silva,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Wanda Silva,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Wanda Silva,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637


In [9]:
df.columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url', 'trava_zap',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'id_persona',
       'message_type', 'messenger', 'media_name', 'media_md5'],
      dtype='object')

In [10]:
ids_para_remover = ['b73c0b674b28d87375d78dcae2f1a21f','62b3aa83e4df122e48cb1f97583e403f',
                    '893874da7344daaa00b8b5b3dc295b59','38737f3c7a07586c4efbfa39aa345705',
                    'da4cbbc1b7b1192ad83bcdae613ac2c3', 'bf66a467459264e0d33e4c8518a82827',
                    '46e009035e6150d656d0c194db88fb07','811541f6ce49347d1798e8dd2d3cbf83',
                    '1d2a0a63333d12ad188e9e6f7f2e0419','7c379613d4da713fe959d8c7ddc11ce2',
                    'debe17fd10d504076d2df8682f63bfc6','e9713ae04a02a810d6f33dd956f42794',
                    'd6c77928db26721ce46aca2d549780f0','8e53464d3b01eea3e39d07e51ecbb1b4',
                    'bacd40da04dd7d13f646993bdcf8e79d','2273d1167a6212812d95dc8fadbae78e',
                    'cd3bb1bdf75be7595e6373171a5c2225','add8c1ba533c5e5450d92c061a5ee7bf',
                    'e198f90df1995528531dd43db0c935ea', '06dc9ac55ed64caab2bd97e9ab717302',
                    'c0110feb539d212836605b66192722dd','f71912700ac5331415408ce229681359',
                    '2ee692357a9c948351c43a9540e859ae', '25b66278176dabe814dfc25a405a2470',
                    '08b21cb1e7de74ef5fe1085230075523']

In [11]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Wanda Silva,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Wanda Silva,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Wanda Silva,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Wanda Silva,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Wanda Silva,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",5,2022-11-16 14:49:39.146502,0.1027,NaN,575796,Wanda Silva,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a
557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,5,2022-11-16 14:49:39.847434,0.0000,NaN,1286443,Wanda Silva,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221
557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",5,2022-11-16 14:49:39.922279,0.0000,NaN,13294,Wanda Silva,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e
557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",5,2022-11-16 14:49:39.992932,0.0000,NaN,1286444,Wanda Silva,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e


In [12]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]



#df_filtrado = df_filtrado[0:100]
len(df_filtrado)

443759

In [13]:
token_espaco = tokenize.WhitespaceTokenizer()
token_pontuacao = tokenize.WordPunctTokenizer()

In [14]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

custom_stopwords = {
    't', 'be', 'nao', 'youtu', 'vai', 'av', 'ja', 'to', 'the', 'this', 'i', 'and', 
    'you', 'y', 'www', 'sao', 'pois', 'contra', 'user', 'ai', 'so', 'gente', 'voce', 'of', 
    'ola', 'gift', 'card', 'kwaivideo', 'r', 'bom', 'q', 'vc', 'vcs', 'pra','ta', 'phone', 'ok', 'la',
    'sera', 'ha', 'aqui', 'ate', 'dia', 'mc', 'im', 'tmj', 'pix', 'g', 'diz', 'ti', 'etc', 'tudo', 
    'todo', 'toda', 'youtube', 'g1', 'm', 'instagram', 'fb', 'in', 'link', 'was', 'blocked', 'kk'
}

stop_words.update(custom_stopwords)

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)

In [16]:
df_geral.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,...,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Wanda Silva,Texto,telegram,NaN,NaN,entao fato renato audio ouvi whatsapp ocorreu ...
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",...,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Wanda Silva,Texto,telegram,NaN,NaN,saiu presidente 8 horas atras infelizmente con...
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",...,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Wanda Silva,Texto,telegram,NaN,NaN,parte quase feita segundo turno completamos pa...
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,...,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Wanda Silva,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,achei seita maconarica
5,2022-10-05 06:28:30,NaN,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,False,Kķkkkkk to rindo até agora....Quem disse q ia ...,...,2022-10-05 06:29:29.046694,0.7003,0.197813,2735,Wanda Silva,Texto,telegram,NaN,NaN,rindo agoraquem disse ia fazer acordo diabo pr...


### Calculando a diversidade dos tópicos
A diversidade de tópicos mede quantas palavras únicas existem entre os top-N termos de todos os tópicos. 

In [17]:

def topic_diversity(model, top_n=10):
    topic_words = []
    for k in range(model.num_topics):
        words = [word for word, _ in model.show_topic(k, topn=top_n)]
        topic_words.extend(words)
    unique_words = set(topic_words)
    return len(unique_words) / (top_n * model.num_topics)


### IRBO 

IRBO avalia quanto os tópicos são distintos entre si, usando o Rank-Biased Overlap (RBO) invertido. Quanto maior o IRBO, mais diferentes são os tópicos

In [18]:
import rbo

def compute_irbo(model, top_n=10):
    topics = []
    for k in range(model.num_topics):
        topic_terms = [word for word, _ in model.show_topic(k, topn=top_n)]
        topics.append(topic_terms)

    # Calcular média dos RBOs invertidos entre todos os pares
    n = len(topics)
    total_irbo = 0
    count = 0

    for i in range(n):
        for j in range(i + 1, n):
            rbo_score = rbo.RankingSimilarity(topics[i], topics[j]).rbo()
            total_irbo += (1 - rbo_score)
            count += 1

    return total_irbo / count if count > 0 else 0


## Modelando os Tópicos sem filtro

In [19]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto]

texto_processado = df_geral['text_processed'].apply(processar_texto)

In [20]:
# Criar dicionário
id2word = corpora.Dictionary(texto_processado)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus = [id2word.doc2bow(text) for text in texto_processado]

In [ ]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus=corpus,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word,
    corpus=corpus,
    texts=texto_processado,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)

# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_v):   0%|          | 0/14 [00:00<?, ?it/s]c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy


Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.3812
Qtd. tópicos = 4 -> Coerência = 0.3789
Qtd. tópicos = 6 -> Coerência = 0.4339
Qtd. tópicos = 8 -> Coerência = 0.4486
Qtd. tópicos = 10 -> Coerência = 0.5081
Qtd. tópicos = 12 -> Coerência = 0.5146
Qtd. tópicos = 14 -> Coerência = 0.519
Qtd. tópicos = 16 -> Coerência = 0.5407
Qtd. tópicos = 18 -> Coerência = 0.4988
Qtd. tópicos = 20 -> Coerência = 0.5316
Qtd. tópicos = 22 -> Coerência = 0.552
Qtd. tópicos = 24 -> Coerência = 0.5688
Qtd. tópicos = 26 -> Coerência = 0.5681
Qtd. tópicos = 28 -> Coerência = 0.539
Melhor número de tópicos segundo C_V: 24

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -6.71
Qtd. tópicos = 4 -> Coerência = -6.1226
Qtd. tópicos = 6 -> Coerência = -5.9185
Qtd. tópicos = 8 -> Coerência = -5.9578
Qtd. tópicos = 10 -> Coerência = -4.788
Qtd. tópicos = 12 -> Coerência = -5.2787
Qtd. tópicos = 14 -> Coerência = -4.4617
Qtd. tópicos = 16 -> Coerência = -4.4887
Qtd. tópicos

In [22]:
# tamanho do corpus
len(corpus)


443759

In [23]:

pyLDAvis.enable_notebook()  


# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.239864  0.070466       1        1  12.964515
16     0.220787 -0.056930       2        1   9.934455
12     0.164119 -0.026031       3        1   7.367259
18     0.119325 -0.160189       4        1   7.112583
9      0.126774  0.183283       5        1   6.124895
11     0.140534 -0.056917       6        1   5.883960
3      0.156101  0.031946       7        1   5.865148
4      0.094750  0.036856       8        1   5.853339
2      0.075889 -0.125323       9        1   5.048294
17    -0.001361 -0.011501      10        1   4.838588
15     0.102563  0.042783      11        1   4.428292
0     -0.216762 -0.144979      12        1   4.360598
8     -0.172836 -0.022575      13        1   3.017697
5     -0.106421  0.249508      14        1   2.883731
7     -0.157034 -0.008248      15        1   2.860243
13    -0.109792 -0.196784      16        1   2.575755
19    -0.186931  0.123628      17        1   2.417352
1     -0.153470 -0.096837      18        1   2.317911
10    -0.125088  0.144730      19        1   2.230833
14    -0.211011  0.023116      20        1   1.914552, topic_info=                  Term          Freq         Total Category  logprob  loglift
73                  13  76516.000000  76516.000000  Default  30.0000  30.0000
755                tse  46662.000000  46662.000000  Default  29.0000  29.0000
745               lula  58492.000000  58492.000000  Default  28.0000  28.0000
617             brazil  40360.000000  40360.000000  Default  27.0000  27.0000
16103        following  39031.000000  39031.000000  Default  26.0000  26.0000
...                ...           ...           ...      ...      ...      ...
745               lula   1155.129396  58492.943916  Topic20  -4.9205   0.0310
57           bolsonaro    953.713196  64793.035154  Topic20  -5.1121  -0.2629
2423               faz    539.662457   6786.743741  Topic20  -5.6815   1.4239
9117                dr    432.713956   2240.875895  Topic20  -5.9023   2.3111
522    tmealexeconomia    403.540438   4991.476575  Topic20  -5.9721   1.4405

[1316 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
7236     20  0.996466   000
67        1  0.004151     1
67        2  0.037269     1
67        3  0.000090     1
67        4  0.094751     1
...     ...       ...   ...
4059      8  0.289663  zona
4059      9  0.000639  zona
4059     10  0.071297  zona
4059     15  0.406039  zona
4059     19  0.027496  zona

[3649 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 17, 13, 19, 10, 12, 4, 5, 3, 18, 16, 1, 9, 6, 8, 14, 20, 2, 11, 15])

In [24]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.835
IRBO médio: 0.9825

Perplexidade:  -10.628525565295552


In [25]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramSemFiltro_u_mass.csv', index=False, encoding='utf-8')


In [26]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'u_mass', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.259452  0.043779       1        1  15.234126
22     0.154505 -0.158696       2        1   7.408228
8      0.217466  0.131061       3        1   6.711550
15     0.112305 -0.085454       4        1   6.470423
16     0.127493 -0.166348       5        1   5.612475
23     0.138363 -0.016940       6        1   5.396398
1      0.120789 -0.004569       7        1   5.259798
6      0.192155  0.044400       8        1   5.251426
7      0.146096  0.205365       9        1   5.130095
10    -0.198715 -0.090757      10        1   4.281093
19    -0.156120 -0.108315      11        1   3.436826
18    -0.070776  0.002243      12        1   3.084662
5      0.001101 -0.200886      13        1   2.989665
9      0.011593 -0.089390      14        1   2.845430
20    -0.110063 -0.064929      15        1   2.827665
4      0.039477  0.098188      16        1   2.589744
13    -0.036205  0.228356      17        1   2.334637
12    -0.145253  0.132056      18        1   2.245050
0     -0.087962 -0.017240      19        1   2.197508
14    -0.048894 -0.036099      20        1   2.136134
3     -0.206000  0.126573      21        1   1.915860
21    -0.102688 -0.073808      22        1   1.645790
2     -0.153067 -0.024413      23        1   1.531080
17    -0.205049  0.125822      24        1   1.464336, topic_info=             Term          Freq         Total Category  logprob  loglift
73             13  68123.000000  68123.000000  Default  30.0000  30.0000
16103   following  56568.000000  56568.000000  Default  29.0000  29.0000
30528   community  46921.000000  46921.000000  Default  28.0000  28.0000
30287    decision  40643.000000  40643.000000  Default  27.0000  27.0000
755           tse  41103.000000  41103.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
739            ar    566.622465   1726.633870  Topic24  -5.3646   3.1095
3799   inscrevase    479.424733    793.820834  Topic24  -5.5318   3.7195
2639      senhora    481.291459    960.779214  Topic24  -5.5279   3.5325
1589        fotos    467.399758   1703.531410  Topic24  -5.5572   2.9305
1259        midia    460.745514   6769.529442  Topic24  -5.5715   1.5364

[1546 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
7214       4  0.143274     0
7214      10  0.026249     0
7214      19  0.829022     0
25924     12  0.997098    00
22269     15  0.002682  0000
...      ...       ...   ...
4059       4  0.168502  zona
4059       5  0.014326  zona
4059       9  0.180100  zona
4059      15  0.519151  zona
4059      19  0.075042  zona

[4364 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 23, 9, 16, 17, 24, 2, 7, 8, 11, 20, 19, 6, 10, 21, 5, 14, 13, 1, 15, 4, 22, 3, 18])

In [27]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.8375
IRBO médio: 0.9778

Perplexidade:  -11.025350422568216


In [28]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramSemFiltro_c_v.csv', index=False, encoding='utf-8')


In [29]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.261318  0.008711       1        1  16.048580
3      0.228735  0.020239       2        1  11.920589
2      0.187826  0.123584       3        1   7.979182
15     0.140905 -0.090277       4        1   7.086191
12     0.154662 -0.111193       5        1   6.365593
0      0.023041 -0.142737       6        1   6.283628
8      0.133450 -0.037652       7        1   5.597673
1      0.093181 -0.169397       8        1   5.303277
16    -0.212757 -0.143648       9        1   4.501261
14     0.014557  0.190316      10        1   4.253841
7      0.020343  0.235543      11        1   3.814538
17    -0.037555  0.120256      12        1   3.614183
5     -0.126002  0.090135      13        1   3.412098
4     -0.133111 -0.173467      14        1   3.191134
13    -0.110634 -0.009482      15        1   3.077432
6     -0.239658  0.117280      16        1   3.070382
10    -0.199043 -0.029568      17        1   2.499479
11    -0.199257  0.001357      18        1   1.980938, topic_info=              Term          Freq         Total Category  logprob  loglift
73              13  69381.000000  69381.000000  Default  30.0000  30.0000
16103    following  52744.000000  52744.000000  Default  29.0000  29.0000
755            tse  41832.000000  41832.000000  Default  28.0000  28.0000
478          grupo  43743.000000  43743.000000  Default  27.0000  27.0000
57       bolsonaro  64892.000000  64892.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
6410      passando    899.197075   2339.335355  Topic18  -5.2050   2.9655
952              p    928.437899   2954.301628  Topic18  -5.1730   2.7641
936         chegar    986.987666   5141.648420  Topic18  -5.1119   2.2711
4444   compartilhe    876.697574  10148.843063  Topic18  -5.2303   1.4726
57       bolsonaro   1025.122911  64892.195162  Topic18  -5.0739  -0.2263

[1185 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
67        1  0.000290     1
67        2  0.139592     1
67        3  0.058501     1
67        4  0.109569     1
67        5  0.140268     1
...     ...       ...   ...
4059     10  0.007719  zona
4059     11  0.001678  zona
4059     12  0.056379  zona
4059     15  0.664137  zona
4059     18  0.024163  zona

[3170 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 3, 16, 13, 1, 9, 2, 17, 15, 8, 18, 6, 5, 14, 7, 11, 12])

In [30]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.8222
IRBO médio: 0.9731

Perplexidade:  -10.365081052922578


In [31]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramSemFiltro_c_uci.csv', index=False, encoding='utf-8')


In [32]:

pyLDAvis.enable_notebook() 

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'u_mass'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.271688  0.045226       1        1  22.632256
5     -0.159475 -0.025503       2        1  12.278868
11    -0.170899 -0.093667       3        1   9.160688
4     -0.205947  0.058458       4        1   8.898410
9     -0.180750 -0.118152       5        1   6.973575
1     -0.151181  0.085040       6        1   5.448675
8      0.210613  0.044813       7        1   4.847903
6      0.099081 -0.239683       8        1   4.803595
0     -0.083669  0.090462       9        1   4.404287
10     0.043065 -0.217648      10        1   3.749975
2      0.089192  0.155670      11        1   3.266219
14     0.105380  0.070773      12        1   3.186978
3      0.115061  0.107195      13        1   2.834564
15     0.236462 -0.185386      14        1   2.580284
7      0.168488  0.197222      15        1   2.508844
12     0.156267  0.025179      16        1   2.424879, topic_info=            Term          Freq         Total Category  logprob  loglift
73            13  67786.000000  67786.000000  Default  30.0000  30.0000
30287   decision  39057.000000  39057.000000  Default  29.0000  29.0000
755          tse  34200.000000  34200.000000  Default  28.0000  28.0000
745         lula  55354.000000  55354.000000  Default  27.0000  27.0000
617       brazil  35376.000000  35376.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1866   eleitoral    515.829800   7234.629475  Topic16  -5.9629   1.0785
5233      acabou    399.113939   3866.750373  Topic16  -6.2195   1.4485
204      digital    388.887646   3205.206830  Topic16  -6.2454   1.6101
4980    internet    341.240178   3269.225247  Topic16  -6.3761   1.4597
432         site    326.555502   3480.347491  Topic16  -6.4201   1.3531

[1035 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
67        1  0.069678     1
67        2  0.331667     1
67        3  0.030562     1
67        4  0.162710     1
67        5  0.040942     1
...     ...       ...   ...
4059      2  0.107617  zona
4059      4  0.011923  zona
4059      5  0.172877  zona
4059      8  0.696842  zona
4059      9  0.009099  zona

[2451 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 6, 12, 5, 10, 2, 9, 7, 1, 11, 3, 15, 4, 16, 8, 13])

In [33]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.8562
IRBO médio: 0.9827

Perplexidade:  -10.159825862020144


In [34]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramSemFiltro_c_npmi.csv', index=False, encoding='utf-8')


## Modelando os Tópicos com filtro religioso

In [35]:
palavras_religiosas = [
    "deus", "jesus", "misericordia", "davi",
    "salomao", "reino", "templo", "conservador",
    "pentecostal", "rcc", "renovacao", "carismatic",
    "paulo ricardo", "bernardo kuster", "herege", "ateu",
    "jerico", "heresia"
]


In [36]:
# Função para verificar se uma palavra está relacionada à religião
def relacionada_religiao(word):
    word_lower = word.lower()
    palavras_religiosas_lower = [palavra.lower() for palavra in palavras_religiosas]

    # Verificando se alguma palavra da lista de palavras religiosas está presente
    return any(palavra in word_lower for palavra in palavras_religiosas_lower)


df_religiao = df_geral[df_geral['text_processed'].apply(lambda x: relacionada_religiao(x))]



In [37]:
len(df_religiao)

25762

In [38]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto]

texto_processado_religiao = df_religiao['text_processed'].apply(processar_texto)

In [39]:
# Criar dicionário
id2word_religiao = corpora.Dictionary(texto_processado_religiao)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus_religiao = [id2word_religiao.doc2bow(text) for text in texto_processado_religiao]

In [ ]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus_religiao, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus_religiao, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus_religiao, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus_religiao=corpus_religiao,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word_religiao,
    corpus_religiao=corpus_religiao,
    texts=texto_processado_religiao,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_npmi): 100%|██████████| 14/14 [06:35<00:00, 28.25s/it]


Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.4625
Qtd. tópicos = 4 -> Coerência = 0.5569
Qtd. tópicos = 6 -> Coerência = 0.4935
Qtd. tópicos = 8 -> Coerência = 0.4985
Qtd. tópicos = 10 -> Coerência = 0.4661
Qtd. tópicos = 12 -> Coerência = 0.4528
Qtd. tópicos = 14 -> Coerência = 0.5243
Qtd. tópicos = 16 -> Coerência = 0.4541
Qtd. tópicos = 18 -> Coerência = 0.4955
Qtd. tópicos = 20 -> Coerência = 0.4546
Qtd. tópicos = 22 -> Coerência = 0.4768
Qtd. tópicos = 24 -> Coerência = 0.4382
Qtd. tópicos = 26 -> Coerência = 0.483
Qtd. tópicos = 28 -> Coerência = 0.4617
Melhor número de tópicos segundo C_V: 4

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -1.5491
Qtd. tópicos = 4 -> Coerência = -1.5581
Qtd. tópicos = 6 -> Coerência = -1.9549
Qtd. tópicos = 8 -> Coerência = -2.4737
Qtd. tópicos = 10 -> Coerência = -2.2193
Qtd. tópicos = 12 -> Coerência = -2.2135
Qtd. tópicos = 14 -> Coerência = -2.6762
Qtd. tópicos = 16 -> Coerência = -2.6864
Qtd. tóp

In [41]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.128796  0.0       1        1  67.292125
0     -0.128796  0.0       2        1  32.707875, topic_info=           Term          Freq         Total Category  logprob  loglift
380       grupo  10572.000000  10572.000000  Default  30.0000  30.0000
236   liberdade   8364.000000   8364.000000  Default  29.0000  29.0000
10         deus  25884.000000  25884.000000  Default  28.0000  28.0000
86           55  11112.000000  11112.000000  Default  27.0000  27.0000
345      amigos   3926.000000   3926.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
131   bolsonaro   5958.864415  11563.770658   Topic2  -4.4114   0.4546
133      brasil   4688.299433  11719.448652   Topic2  -4.6512   0.2014
408  presidente   3145.924898   6421.138137   Topic2  -5.0502   0.4041
414       todos   3509.723826   9134.712997   Topic2  -4.9408   0.1610
940       vamos   2326.102203   5264.052770   Topic2  -5.3521   0.3008

[197 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
60237      1  0.008619  15112022
60237      2  0.991143  15112022
1811       1  0.036788       200
1811       2  0.963187       200
49         1  0.274011        22
...      ...       ...       ...
334        2  0.105376      vida
5159       1  0.004936    vindoa
5159       2  0.995390    vindoa
3983       1  0.005003   votarao
3983       2  0.990599   votarao

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [42]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.8
IRBO médio: 0.548

Perplexidade:  -8.428196276928846


In [43]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroReligioso_u_mass.csv', index=False, encoding='utf-8')


In [44]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'u_mass', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.063123  0.042184       1        1  47.049167
3     -0.103552  0.118857       2        1  21.956560
2     -0.077435 -0.168817       3        1  20.500538
0      0.244109  0.007777       4        1  10.493735, topic_info=           Term          Freq         Total Category  logprob  loglift
86           55  10441.000000  10441.000000  Default  30.0000  30.0000
236   liberdade   7831.000000   7831.000000  Default  29.0000  29.0000
380       grupo  10014.000000  10014.000000  Default  28.0000  28.0000
192     familia   5510.000000   5510.000000  Default  27.0000  27.0000
10         deus  26470.000000  26470.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3398      paulo   1360.878204   2125.315396   Topic4  -4.7513   1.8086
4           bem   2866.063794   5714.734972   Topic4  -4.0065   1.5643
2205    janeiro    688.933958   1003.205733   Topic4  -5.4321   1.8786
10         deus   4148.954224  26470.984860   Topic4  -3.6366   0.4012
298         rio    706.262301   1311.061408   Topic4  -5.4072   1.6358

[355 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
20333      1  0.965484        0xx47
20333      2  0.026307        0xx47
20333      3  0.002631        0xx47
20333      4  0.005261        0xx47
61373      4  0.931679  11946464824
...      ...       ...          ...
3983       3  0.993108      votarao
897        1  0.396039        votos
897        2  0.033995        votos
897        3  0.558647        votos
897        4  0.011332        votos

[943 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1])

In [45]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.725
IRBO médio: 0.6888

Perplexidade:  -8.268320857515386


In [46]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroReligioso_c_v.csv', index=False, encoding='utf-8')


In [47]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.050746  0.101161       1        1  40.828725
1     -0.081517  0.124765       2        1  27.415627
2     -0.121324 -0.193635       3        1  18.592682
0      0.253587 -0.032291       4        1  13.162966, topic_info=           Term          Freq         Total Category  logprob  loglift
86           55  13752.000000  13752.000000  Default  30.0000  30.0000
236   liberdade   7802.000000   7802.000000  Default  29.0000  29.0000
380       grupo  10186.000000  10186.000000  Default  28.0000  28.0000
10         deus  26609.000000  26609.000000  Default  27.0000  27.0000
271      patria   4637.000000   4637.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
380       grupo   5515.700163  10186.526444   Topic4  -3.5785   1.4143
3398      paulo   1424.775172   2073.598349   Topic4  -4.9321   1.6525
4           bem   2969.551517   5627.718811   Topic4  -4.1977   1.3885
10         deus   6304.393436  26609.718589   Topic4  -3.4448   0.5877
686      porque    907.552264   4317.302908   Topic4  -5.3831   0.4681

[386 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
42492      1  0.989792     0000
40025      1  0.009570      001
40025      2  0.014355      001
40025      3  0.004785      001
40025      4  0.971343      001
...      ...       ...      ...
2361       3  0.024962    voces
2361       4  0.012481    voces
20573      4  0.985987  vomitar
3983       3  0.992442  votarao
3983       4  0.005279  votarao

[1012 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1])

In [48]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.725
IRBO médio: 0.8095

Perplexidade:  -8.216203263859684


In [49]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroReligioso_c_uci.csv', index=False, encoding='utf-8')


In [50]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.117525  0.0       1        1  68.706887
1     -0.117525  0.0       2        1  31.293113, topic_info=           Term          Freq         Total Category  logprob  loglift
86           55  10908.000000  10908.000000  Default  30.0000  30.0000
236   liberdade   7787.000000   7787.000000  Default  29.0000  29.0000
380       grupo  10018.000000  10018.000000  Default  28.0000  28.0000
2338       lula   6397.000000   6397.000000  Default  27.0000  27.0000
345      amigos   3594.000000   3594.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
4           bem   3360.966444   5619.937283   Topic2  -4.9398   0.6477
192     familia   3337.473333   5580.820156   Topic2  -4.9469   0.6477
10         deus   8302.796380  26012.277815   Topic2  -4.0355   0.0198
133      brasil   4107.426772  11384.858603   Topic2  -4.7393   0.1423
131   bolsonaro   1667.721708  11958.891291   Topic2  -5.6406  -0.8082

[210 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
20333      1  0.991757     0xx47
20333      2  0.008405     0xx47
11246      1  0.991173       142
11246      2  0.009079       142
60237      1  0.997563  15112022
...      ...       ...       ...
2790       2  0.042945   vitoria
2361       1  0.929151     voces
2361       2  0.071079     voces
7334       1  0.008008   welcome
7334       2  0.992988   welcome

[347 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [51]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.9
IRBO médio: 0.7425

Perplexidade:  -8.452552014271284


In [52]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroReligioso_c_npmi.csv', index=False, encoding='utf-8')


## Modelando os tópicos com filtro de inclusão religioso e exclusão de termas políticos

In [53]:
palavras_politicas = [ "lula", "bolsonaro", "pt", "pl", "stf", "patria", "55", "22", "13", "senadores", "lulaladrao",
                       "urnas", "alexandre", "moraes", "comunismo", "eleicao", "eleicoes", "esquerda", "direita",
                         "presidente", "tse", "fraude", "voto", "turno", "ministro"]

In [54]:
def retirar_mensagens_com_palavras_politicas(word):
    word_lower = word.lower()
    palavras_politica_lower = [palavra.lower() for palavra in palavras_politicas]

    return any(palavra in word_lower for palavra in palavras_politica_lower)


df_politico = df_religiao[~df_religiao['text_processed'].apply(lambda x: retirar_mensagens_com_palavras_politicas(x))]



In [55]:
df_politico.shape

(8526, 21)

In [56]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto ]

texto_processado_politico = df_politico['text_processed'].apply(processar_texto)

In [57]:
# Criar dicionário
id2word_politico = corpora.Dictionary(texto_processado_politico)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus_politico = [id2word_politico.doc2bow(text) for text in texto_processado_politico]

In [ ]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus_politico, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus_politico, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus_politico, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus_politico=corpus_politico,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word_politico,
    corpus_politico=corpus_politico,
    texts=texto_processado_politico,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_npmi): 100%|██████████| 14/14 [05:23<00:00, 23.10s/it]


Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.514
Qtd. tópicos = 4 -> Coerência = 0.418
Qtd. tópicos = 6 -> Coerência = 0.4135
Qtd. tópicos = 8 -> Coerência = 0.4168
Qtd. tópicos = 10 -> Coerência = 0.4411
Qtd. tópicos = 12 -> Coerência = 0.5029
Qtd. tópicos = 14 -> Coerência = 0.4647
Qtd. tópicos = 16 -> Coerência = 0.45
Qtd. tópicos = 18 -> Coerência = 0.4957
Qtd. tópicos = 20 -> Coerência = 0.4633
Qtd. tópicos = 22 -> Coerência = 0.4537
Qtd. tópicos = 24 -> Coerência = 0.4311
Qtd. tópicos = 26 -> Coerência = 0.4623
Qtd. tópicos = 28 -> Coerência = 0.4536
Melhor número de tópicos segundo C_V: 2

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -3.9611
Qtd. tópicos = 4 -> Coerência = -6.2067
Qtd. tópicos = 6 -> Coerência = -5.6495
Qtd. tópicos = 8 -> Coerência = -6.3218
Qtd. tópicos = 10 -> Coerência = -4.3434
Qtd. tópicos = 12 -> Coerência = -5.3282
Qtd. tópicos = 14 -> Coerência = -5.6359
Qtd. tópicos = 16 -> Coerência = -5.6224
Qtd. tópico

In [59]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.103135  0.0       1        1  57.313122
0     -0.103135  0.0       2        1  42.686878, topic_info=           Term         Freq        Total Category  logprob  loglift
58         deus  7080.000000  7080.000000  Default  30.0000  30.0000
19521   repitam   888.000000   888.000000  Default  29.0000  29.0000
951      existe   962.000000   962.000000  Default  28.0000  28.0000
7        comigo  1113.000000  1113.000000  Default  27.0000  27.0000
257    obrigado   880.000000   880.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
212        vida   310.155341   604.637246   Topic2  -5.5692   0.1837
54       cristo   275.550842   529.877738   Topic2  -5.6875   0.1974
220        amor   266.671828   504.781676   Topic2  -5.7203   0.2132
638         ter   244.594634   384.078749   Topic2  -5.8067   0.4000
341        povo   232.079524   539.814469   Topic2  -5.8592   0.0071

[188 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.979843       29
0         2  0.017497       29
1009      1  0.979904        6
1009      2  0.018146        6
6557      1  0.990048      832
...     ...       ...      ...
212       2  0.512704     vida
1912      1  0.038776  welcome
1912      2  0.956469  welcome
3684      1  0.024944     yhvh
3684      2  0.972808     yhvh

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [60]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.8
IRBO médio: 0.5428

Perplexidade:  -8.36552140871751


In [61]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroPolitico_u_mass.csv', index=False, encoding='utf-8')


In [62]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.095318  0.0       1        1  50.793793
1     -0.095318  0.0       2        1  49.206207, topic_info=           Term         Freq        Total Category  logprob  loglift
7        comigo  1138.000000  1138.000000  Default  30.0000  30.0000
951      existe   983.000000   983.000000  Default  29.0000  29.0000
19521   repitam   913.000000   913.000000  Default  28.0000  28.0000
257    obrigado  1120.000000  1120.000000  Default  27.0000  27.0000
193      senhor   915.000000   915.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
3203      maria   236.283839   323.277662   Topic2  -5.9834   0.3957
413       feliz   238.220823   337.832101   Topic2  -5.9752   0.3598
250     abencoe   311.097140   958.017170   Topic2  -5.7083  -0.4156
220        amor   257.616260   515.700037   Topic2  -5.8969   0.0151
138       mundo   237.151058   348.158675   Topic2  -5.9797   0.3252

[198 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
1363       1  0.036362       24
1363       2  0.963589       24
17001      1  0.020909      2cr
17001      2  0.982725      2cr
14325      1  0.020450      714
...      ...       ...      ...
10573      2  0.033493  virtual
2112       1  0.950902    vivos
2112       2  0.051750    vivos
1912       1  0.968330  welcome
1912       2  0.038224  welcome

[323 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [63]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.8
IRBO médio: 0.5846

Perplexidade:  -8.436786156963084


In [64]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroPolitico_c_v.csv', index=False, encoding='utf-8')


In [65]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.077787  0.0       1        1  57.032402
1     -0.077787  0.0       2        1  42.967598, topic_info=          Term         Freq        Total Category  logprob  loglift
58        deus  7736.000000  7736.000000  Default  30.0000  30.0000
19521  repitam  1223.000000  1223.000000  Default  29.0000  29.0000
951     existe  1309.000000  1309.000000  Default  28.0000  28.0000
7       comigo  1411.000000  1411.000000  Default  27.0000  27.0000
528      jesus  1626.000000  1626.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
341       povo   264.645766   614.715804   Topic2  -5.7344   0.0020
638        ter   209.311465   367.993942   Topic2  -5.9690   0.2805
779      acima   219.451590   435.289369   Topic2  -5.9217   0.1598
1064     agora   230.515786   566.091122   Topic2  -5.8725  -0.0537
160      poder   212.940823   483.739254   Topic2  -5.9518   0.0242

[209 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
18190      1  0.080963  11983308937
18190      2  0.971557  11983308937
3521       1  0.974752          317
3521       2  0.027850          317
3522       1  0.968698  41161774823
...      ...       ...          ...
1912       2  0.016670      welcome
5060       1  0.062694         west
5060       2  0.940405         west
3684       1  0.976495         yhvh
3684       2  0.036849         yhvh

[340 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [66]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.85
IRBO médio: 0.6089

Perplexidade:  -8.307783796321262


In [67]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroPolitico_c_uci.csv', index=False, encoding='utf-8')


In [68]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.041311 -0.003405       1        1  10.501726
15     0.029255 -0.001949       2        1   9.488515
2      0.037412 -0.082069       3        1   9.214814
4     -0.111142 -0.089848       4        1   8.475324
11     0.028877  0.088623       5        1   5.500332
0      0.035840  0.038592       6        1   5.149280
19     0.044406  0.094328       7        1   5.046839
16    -0.224002 -0.101372       8        1   4.720132
1     -0.002188  0.075910       9        1   4.452439
10    -0.015951  0.033945      10        1   4.123390
6      0.017602  0.106280      11        1   3.864453
8     -0.196691 -0.018745      12        1   3.787055
3     -0.199672 -0.042259      13        1   3.379379
18     0.159539 -0.183102      14        1   3.330264
9      0.022255  0.081552      15        1   3.322122
7      0.142800 -0.234707      16        1   3.258740
17     0.062866  0.106894      17        1   3.169687
14     0.008174  0.104882      18        1   3.160868
13     0.072637  0.048417      19        1   3.030302
12     0.046672 -0.021967      20        1   3.024340, topic_info=          Term         Freq        Total Category  logprob  loglift
951     existe  1210.000000  1210.000000  Default  30.0000  30.0000
19521  repitam  1132.000000  1132.000000  Default  29.0000  29.0000
7       comigo  1350.000000  1350.000000  Default  28.0000  28.0000
58        deus  8322.000000  8322.000000  Default  27.0000  27.0000
240       nada   839.000000   839.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
588       deve    23.222237    79.071590  Topic20  -5.5140   2.2732
638        ter    24.494175   335.132264  Topic20  -5.4606   0.8824
455    sistema    23.228639   116.276244  Topic20  -5.5137   1.8879
138      mundo    23.475627   327.870943  Topic20  -5.5031   0.8618
154        pai    23.501574   393.657494  Topic20  -5.5020   0.6801

[1241 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
21972     13  0.926636   0029
15895      1  0.985884   1034
2653       2  0.038730     14
2653       6  0.038730     14
2653      10  0.890786     14
...      ...       ...    ...
1444       5  0.247732   yhwh
1444       6  0.594556   yhwh
1444      11  0.099093   yhwh
21853     11  0.962756  yuval
21974     13  0.926541     zp

[4225 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 16, 3, 5, 12, 1, 20, 17, 2, 11, 7, 9, 4, 19, 10, 8, 18, 15, 14, 13])

In [69]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.685
IRBO médio: 0.8047

Perplexidade:  -8.532287929346715


In [70]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/telegram/lda_model_telegramComFiltroPolitico_c_npmi.csv', index=False, encoding='utf-8')
